In [1]:
import random
from pathlib import Path
import numpy as np
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from sklearn.model_selection import train_test_split
from processing.corpus import Corpus

SEED = np.random.seed(0)
DATA = Path('data')
TARGETS = ['participants', 'interventions', 'outcomes']
SUBSET = 'Train' # 'Test'
N_TRAIN_DOCS = 4000
SAVE_LOC = Path('data\\split')

SAVE_LOC.mkdir(exist_ok=True, parents=True)

%load_ext jupyternotify

<IPython.core.display.Javascript object>

# Load feature set and labels

In [2]:
labels_path = DATA / 'raw' / 'labels.parquet'
feats_in = pd.read_parquet(DATA / 'dataset3.parquet')
labels_in = pd.read_parquet(labels_path).drop('Word', axis=1)
      
df = pd.concat([c.reset_index(drop=True) for c in [feats_in, labels_in]], axis=1).set_index(feats_in.index)

assert not df.columns.duplicated().any()
      
print('Data load complete.')

Data load complete.


In [3]:
# downsample = True
# if downsample:
#     n = 2500
#     doc_ids = list(df.index.unique('doc'))
#     sample_idx = random.sample(doc_ids, n)

#     df = df.loc[(sample_idx, slice(None), slice(None)), :]

# Train/Test split

Withhold part of the data for evaluation. Could update using the same method as in GBC script, as it seems to be faster.

In [4]:
k_folds = 10 # only simulated for now

train_idx, val_idx = train_test_split(df.index.unique('doc'), train_size=1-(1/k_folds)) # default 80:20

print(f'splitting data {len(train_idx)}:{len(val_idx)}')

train_val = df.loc[(train_idx, slice(None)),:]
test_hold = df.loc[(val_idx, slice(None)),:]

splitting data 4493:500


In [5]:
doc_ids = list(df.index.unique(level='doc'))
n_docs = [4000]

train_ds = [train_val.loc[random.sample(doc_ids, n)] for n in n_docs]


train_val.to_parquet(SAVE_LOC / 'train.parquet')
test_hold.to_parquet(SAVE_LOC / 'test.parquet')
            
# for i, data in enumerate(train_ds):
#     data.to_parquet(SAVE_LOC / f'train_{n_docs[i]}.parquet')

In [6]:
train_val['lemma']

doc       sent  word
10036953  1     1               _
                2               _
                3         therapy
                4         regimen
                5               _
                          ...    
9989713   15    21      randomise
                22              _
                23       clinical
                24          trial
                25              _
Name: lemma, Length: 1219236, dtype: category
Categories (899, object): [30, 60, _, abdominal, ..., xDECx, xNUMx, year, young]